# CSV to RDF Conversion Example

CSV is by far the most common format of "open data" on the web. This small tutorial shows how CSV can be converted to RDF in a programmatic fashion. Other tools exist, such as [OpenRefine](http://openrefine.org) with the [RDF extension](https://github.com/fadmaa/grefine-rdf-extension/releases) or [LODRefine](https://github.com/sparkica/LODRefine) (which has RDF support built-in) but they do not really scale beyond simple datasets, and do not offer a repeatable framework (as e.g. can be used in a conversion pipeline).

### Reading a CSV file

Make sure the file `example.csv` is in the same directory as this IPython notebook.

In [ ]:
import csv

filename = "example.csv"

with open(filename,'r') as csvfile:
    # Set the right quote character and delimiter
    csvreader = csv.reader(csvfile,quotechar='"',delimiter=';')
    
    # If the first row contains header information, we can retrieve it like so:
    header = csvreader.next()
    print "Header"
    print header
    
    print "Lines"
    for line in csvreader:
        # Line is an array of the columns in the file
        # Make sure to check the encoding of the strings in the array... this often causes issues
        print line

Alternatively, you can use a `csv.DictReader` object to turn the entire CSV file into a list of dictionaries. Note that this will load the CSV file into memory. For large CSV files, it is better to process the file line by line.

In [ ]:
from csv import DictReader

filename = "example.csv"

with open(filename,'r') as csvfile:
    csv_contents = [{k: v for k, v in row.items()}
        for row in csv.DictReader(csvfile, skipinitialspace=True, quotechar='"', delimiter=';')]
    
print csv_contents

### Setting up stuff for RDF

We import the things we'll need from `rdflib`:

* `Dataset` is the object in which we will store our RDF graphs
* `URIRef` is the datatype for URI-resources
* `Literal` is the datatype for literal resources (strings, dates etc.)
* `Namespace` is used to create namespaces (parts of the URI's we are going to make)
* `RDF`, `RDFS`, `OWL` and `XSD` are built in namespaces

**NB**: We'll use "group 20" for this example, but you should replace it with your own group name!

In [ ]:
from rdflib import Dataset, URIRef, Literal, Namespace, RDF, RDFS, OWL, XSD

# A namespace for our resources
data = 'http://data.krw.d2s.labs.vu.nl/group20/resource/'
DATA = Namespace(data)
# A namespace for our vocabulary items (schema information, RDFS, OWL classes and properties etc.)
vocab = 'http://data.krw.d2s.labs.vu.nl/group20/vocab/'
VOCAB = Namespace('http://data.krw.d2s.labs.vu.nl/group20/vocab/')

# The URI for our graph
graph_uri = URIRef('http://data.krw.d2s.labs.vu.nl/group20/resource/examplegraph')

# We initialize a dataset, and bind our namespaces
dataset = Dataset()
dataset.bind('g20data',DATA)
dataset.bind('g20vocab',VOCAB)

# We then get a new graph object with our URI from the dataset.
graph = dataset.graph(graph_uri)

### Let's make some RDF from our CSV Dictionary

A straightforward conversion:

* Make sure you have URIRef objects for all resources you want to make
* Make sure you have Literal objects for all literal values you need. Be sure to use the proper datatype or a language tag.
* Decide on what URI will be the 'primary key' for each row.
* Decide on the terms you are going to use to create the relations (predicates, properties)
* Add the triples to the graph

In [ ]:
# IRI baker is a library that reliably creates valid (parts of) IRIs from strings (spaces are turned into underscores, etc.).
from iribaker import to_iri

# Let's iterate over the dictionary, and create some triples
# Let's pretend we know exactly what the 'schema' of our CSV file is
for row in csv_contents:
    # `Name` is the primary key and we use it as our primary resource, but we'd also like to use it as a label
    person = URIRef(to_iri(data + row['Name']))
    name = Literal(row['Name'], datatype=XSD['string'])
    # `Country` is a resource
    country = URIRef(to_iri(data + row['Country']))
    # But we'd also like to use the name as a label (with a language tag!)
    country_name = Literal(row['Country'], lang='en')
    # `Age` is a literal (an integer)
    age = Literal(int(row['Age']), datatype=XSD['int'])
    # `Favourite Colour` is a resource
    colour = URIRef(to_iri(data + row['Favourite Colour']))
    colour_name = Literal(row['Favourite Colour'], lang='en')
    # `Place` is a resource
    place = URIRef(to_iri(data+ row['Place']))
    place_name = Literal(row['Place'], lang='en')
    # `Address` is a literal (a string)
    address = Literal(row['Address'], datatype=XSD['string'])
    # `Hobby` is a resource
    hobby = URIRef(to_iri(data + row['Hobby']))
    hobby_name = Literal(row['Hobby'], lang='en')
    
    # All set... we are now going to add the triples to our graph
    graph.add((person, VOCAB['name'], name))
    graph.add((person, VOCAB['age'], age))
    graph.add((person, VOCAB['address'], address))
    
    # Add the place and its label
    graph.add((person, VOCAB['place'], place))
    graph.add((place, VOCAB['name'], place_name))
    
    # Add the country and its label
    graph.add((person, VOCAB['country'], country))
    graph.add((country, VOCAB['name'], country_name))
    
    # Add the favourite colour and its label
    graph.add((person, VOCAB['favourite_colour'], colour))
    graph.add((colour, VOCAB['name'], colour_name))
    
    # Add the hobby and its label
    graph.add((person, VOCAB['hobby'], hobby))
    graph.add((hobby, VOCAB['name'], hobby_name))


Let's see how this turned out:

In [ ]:
print dataset.serialize(format='trig')

### Saving the RDF to a file

In [ ]:
with open('example-simple.trig','w') as f:
    graph.serialize(f, format='trig')

### Now, with a bit more thought

Actually, we were a bit naive just now.

* We are implicitly defining a schema: all property names are schema information. We might want to include an (externally defined) schema information.
* Some of these properties may have useful standard names (e.g. for the `g20vocab:name` property we can use `rdfs:label`).
* We have not specified types for our URIs.
* The CSV-file specific named graph may not be the best place for some of our information (e.g. the names of things that may occur in multiple graphs).
* And we have 2 *different* Amsterdam resources... one in NL, the other one in the US.

In [ ]:
# Clear the graph from the dataset (because we're going to start anew)
dataset.remove_graph(graph)
# And get a new object (with the same URI, to create some confusion)
graph = dataset.graph(graph_uri)


# Load the externally defined schema into the default graph (context) of the dataset
dataset.default_context.parse('vocab.ttl', format='turtle')


# Let's iterate over the dictionary, and create some triples
# Let's pretend we know exactly what the 'schema' of our CSV file is
for row in csv_contents:
    # `Name` is the primary key and we use it as our primary resource, but we'd also like to use it as a label
    person = URIRef(to_iri(data + row['Name']))
    name = Literal(row['Name'], datatype=XSD['string'])
    # `Country` is a resource
    country = URIRef(to_iri(data + row['Country']))
    # But we'd also like to use the name as a label (with a language tag!)
    country_name = Literal(row['Country'], lang='en')
    # `Age` is a literal (an integer)
    age = Literal(int(row['Age']), datatype=XSD['int'])
    # `Favourite Colour` is a resource
    colour = URIRef(to_iri(data + row['Favourite Colour']))
    colour_name = Literal(row['Favourite Colour'], lang='en')
    # `Place` is a resource, but we are now going to prepend the country to avoid ambiguity
    place = URIRef(to_iri(data + row['Country'] + '/' + row['Place']))
    place_name = Literal(row['Place'], lang='en')
    # `Address` is a literal (a string)
    address = Literal(row['Address'], datatype=XSD['string'])
    # `Hobby` is a resource
    hobby = URIRef(to_iri(data + row['Hobby']))
    hobby_name = Literal(row['Hobby'], lang='en')
    
    # All set... we are now going to add the triples to our graph
    graph.add((person, RDFS.label, name))
    graph.add((person, VOCAB['age'], age))
    graph.add((person, VOCAB['address'], address))
    
    # Add the place, its label and its type.
    graph.add((person, VOCAB['place'], place))
    dataset.add((place, RDFS.label, place_name))
    dataset.add((place, RDF.type, VOCAB['Place']))
    
    # Add the country and its label
    graph.add((person, VOCAB['country'], country))
    dataset.add((country, RDFS.label, country_name))
    dataset.add((country, RDF.type, VOCAB['Country']))
    
    # Add the favourite colour and its label
    graph.add((person, VOCAB['favourite_colour'], colour))
    dataset.add((colour, RDFS.label, colour_name))
    dataset.add((colour, RDF.type, VOCAB['Colour']))
    
    # Add the hobby and its label
    graph.add((person, VOCAB['hobby'], hobby))
    dataset.add((hobby, RDFS.label, hobby_name))
    dataset.add((hobby, RDF.type, VOCAB['Hobby']))



Let's see:

In [ ]:
print dataset.serialize(format='trig')

### ... and Save the RDF to a file

In [ ]:
with open('example-extended.trig','w') as f:
    dataset.serialize(f, format='trig')

### But for the smartypants...

We can actually do without a lot of the explicit assertion of types, since the domain and range definitions already provide us with the miminal information needed to infer the types of all of these things.

Let's run the conversion again, without adding the types, and inspect the outcome in TopBraid (or Stardog).

Since we've been adding stuff to the default graph, we have to re-initialize our dataset entirely.

In [ ]:
# We initialize a fresh dataset, and bind our namespaces
dataset = Dataset()
dataset.bind('g20data',DATA)
dataset.bind('g20vocab',VOCAB)

# And get a new object (with the same URI, to create some confusion)
graph = dataset.graph(graph_uri)


# Load the externally defined schema into the default graph (context) of the dataset
dataset.default_context.parse('vocab.ttl', format='turtle')


# Let's iterate over the dictionary, and create some triples
# Let's pretend we know exactly what the 'schema' of our CSV file is
for row in csv_contents:
    # `Name` is the primary key and we use it as our primary resource, but we'd also like to use it as a label
    person = URIRef(to_iri(data + row['Name']))
    name = Literal(row['Name'], datatype=XSD['string'])
    # `Country` is a resource
    country = URIRef(to_iri(data + row['Country']))
    # But we'd also like to use the name as a label (with a language tag!)
    country_name = Literal(row['Country'], lang='en')
    # `Age` is a literal (an integer)
    age = Literal(int(row['Age']), datatype=XSD['int'])
    # `Favourite Colour` is a resource
    colour = URIRef(to_iri(data + row['Favourite Colour']))
    colour_name = Literal(row['Favourite Colour'], lang='en')
    # `Place` is a resource, but we are now going to prepend the country to avoid ambiguity
    place = URIRef(to_iri(data + row['Country'] + '/' + row['Place']))
    place_name = Literal(row['Place'], lang='en')
    # `Address` is a literal (a string)
    address = Literal(row['Address'], datatype=XSD['string'])
    # `Hobby` is a resource
    hobby = URIRef(to_iri(data + row['Hobby']))
    hobby_name = Literal(row['Hobby'], lang='en')
    
    # All set... we are now going to add the triples to our graph
    graph.add((person, RDFS.label, name))
    graph.add((person, VOCAB['age'], age))
    graph.add((person, VOCAB['address'], address))
    
    # Add the place, its label and its type.
    graph.add((person, VOCAB['place'], place))
    dataset.add((place, RDFS.label, place_name))
    
    # Add the country and its label
    graph.add((person, VOCAB['country'], country))
    dataset.add((country, RDFS.label, country_name))
    
    # Add the favourite colour and its label
    graph.add((person, VOCAB['favourite_colour'], colour))
    dataset.add((colour, RDFS.label, colour_name))
    
    # Add the hobby and its label
    graph.add((person, VOCAB['hobby'], hobby))
    dataset.add((hobby, RDFS.label, hobby_name))



Check that indeed the types are missing from the named graph:

In [ ]:
print dataset.serialize(format='trig')

### And Save the RDF again...

In [ ]:
with open('example-inferencing.trig','w') as f:
    dataset.serialize(f, format='trig')

# Connecting to Stardog

We now upload the RDF to Stardog, to see how this inferencing thing works (we're using the same code as used in the [Web Application tutorial](https://github.com/KRontheWeb/web-application)

In [ ]:
import requests

TUTORIAL_REPOSITORY = "http://stardog.krw.d2s.labs.vu.nl/group20"

def upload_to_stardog(data):
    transaction_begin_url = TUTORIAL_REPOSITORY + "/transaction/begin"
    
    # Start the transaction, and get a transaction_id
    response = requests.post(transaction_begin_url, headers={'Accept': 'text/plain'})
    transaction_id = response.content

    # POST the data to the transaction
    post_url = TUTORIAL_REPOSITORY + "/" + transaction_id + "/add"
    response = requests.post(post_url, data=data, headers={'Accept': 'text/plain', 'Content-type': 'application/trig'})

    # Close the transaction
    transaction_close_url = TUTORIAL_REPOSITORY + "/transaction/commit/" + transaction_id
    response = requests.post(transaction_close_url)

    return str(response.status_code)

# Upload the serialization of our dataset to Stardog
upload_to_stardog(dataset.serialize(format='trig'))

And let's see whether we can query Stardog for the 'inferred' information.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

query = """
PREFIX gr20vocab: <http://data.krw.d2s.labs.vu.nl/group20/vocab/> 

SELECT * WHERE {
  ?x a gr20vocab:Person .
}"""

endpoint = TUTORIAL_REPOSITORY + '/query'

sparql = SPARQLWrapper(endpoint)

sparql.setQuery(query)

sparql.setReturnFormat(JSON)
sparql.addParameter('Accept','application/sparql-results+json')

Without inferencing:

In [ ]:
sparql.addParameter('reasoning','false')
response = sparql.query().convert()
print response

With inferencing

In [ ]:
sparql.addParameter('reasoning','true')
response = sparql.query().convert()
print response

# Fin